In [1]:
# import
import glob
from pathlib import Path
import win32com.client
from win32com.client import Dispatch
import pandas as pd
import duckdb
import re
from pdfminer.high_level import extract_text
import pikepdf
import fitz
from pretty_html_table import build_table
import random
from datetime import datetime
import time

In [2]:
# fetch LCs
def fetch_read_lc(rec_date_from, rec_date_to): 

    # output folder
    output_dir = Path.cwd() / 'Emailed LCs'
    output_dir.mkdir(parents=True, exist_ok=True)

    # outlook inbox
    outlook = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
    inbox = outlook.Folders.Item(1).Folders['Trade Alerts']

    # emails
    messages = inbox.Items
    for message in reversed(messages): 

        # time
        try: rec_date = str(message.SentOn)[0:10]
        except: continue
        if rec_date < rec_date_from or rec_date > rec_date_to: continue

        # attachments
        attachments = message.Attachments
        for attachment in attachments:
            
            # LCs
            filename = attachment.FileName
            if re.match(".+-T02.PDF", filename): attachment.SaveAsFile(output_dir / "HSBC LCs" / filename)      
            if re.match("^[0-9]+.pdf$", filename): attachment.SaveAsFile(output_dir / "SCB LCs" / filename)     
            if re.match(".+ACK.pdf", filename, re.IGNORECASE): attachment.SaveAsFile(output_dir / "MTB LCs" / filename)
            if re.match(".+SWIFT.+.pdf", filename, re.IGNORECASE): attachment.SaveAsFile(output_dir / "BBL LCs" / filename)
            if re.match(".+700.pdf", filename, re.IGNORECASE): attachment.SaveAsFile(output_dir / "PRB LCs" / filename)


In [3]:
# HSBC
def parse_hsbc(f):

    # fetch datapoint
    def get_data_btn(text_str, sub1, sub2):
        text_str = text_str.replace(sub1, "*").replace(sub2, "*")
        datapoint = text_str.split("*")[1].strip()
        return datapoint

    # scrape
    pdf_name = []
    dc_no = []
    dc_curr = []
    dc_amt = []
    beneficiary = []
    issue_date = []
    insurance_no = []
    insurance_date = []
    material_name = []
    payment_term = []
    bb_ref = []
    inv_no = []
    hs_code_importer = []
    hs_code_exporter = []
    hs_code = []
    upas_tenor_1 = []
    upas_tenor_2 = []
    
    # doc name
    val = f.split("\\")[-1]
    print("Parsing: " + val)
    pdf_name.append(val)

    # all text
    text = extract_text(f)
    text = text.replace("\n", " ")
    text = re.sub("Page:\d+\s/\s\d+", " ", text)
    text = re.sub(" +", " ", text)

    # dc no.
    val = get_data_btn(text, "DC NO:", "DATE OF ISSUE:")
    dc_no.append(val)

    # dc value
    val = get_data_btn(text, "DC AMT: ", " AVAILABLE WITH/BY")
    # currency
    pattern = re.compile("^[A-Z]*")
    dc_curr.append(pattern.findall(val)[0])
    # amount
    pattern = re.compile("[0-9,]+")
    dc_amt.append(pattern.findall(val)[0].strip())

    # beneficiary
    val = get_data_btn(text, "BENEFICIARY: ", "DC AMT:")
    pattern = re.compile(".+(?:LTD|LIMITED)")
    try: beneficiary.append(pattern.findall(val)[0])
    except: beneficiary.append(val)

    # issue date
    val = get_data_btn(text, "DATE OF ISSUE:", "APPLICABLE RULES:")
    issue_date.append(val)

    # insurance date
    val = get_data_btn(text, "MENTIONING INSURANCE", "WITHIN")
    insurance_date.append(val.split()[-1])

    # insurance no.
    pattern = re.compile("UIC[\w/\-\(\s]+[\)0-9]\s")
    insurance_no.append(pattern.findall(val)[0].strip())

    # BB ref
    pattern = re.compile("BANK DC NO.\s*\d+")
    try: bb_ref.append(pattern.findall(text)[0])
    except: bb_ref.append(None)

    # invoice no. 
    val = get_data_btn(text, "GOODS: ", "DOCUMENTS REQUIRED: ")
    pattern = re.compile("(?:INDENT|INVOICE) NO.+\d\sDATE")
    try: inv_no.append(pattern.findall(val)[0][:-5])
    except: inv_no.append(None)

    # payment term
    pattern = re.compile(".+\+")
    try: payment_term.append(pattern.findall(val)[0][:-2])
    except: payment_term.append(None)

    # HS code - importer
    pattern = re.compile("\d+\.\d+\.\d+\s*\(IMP")
    try: hs_code_importer.append(pattern.findall(text)[0].split("(")[0].strip())
    except: hs_code_importer.append(None)

    # HS code - exporter
    pattern = re.compile("\d+\.\d+\.\d+\s*\(EXP")
    try: hs_code_exporter.append(pattern.findall(text)[0].split("(")[0].strip())
    except: hs_code_exporter.append(None)

    # HS code
    pattern = re.compile("\d+\.\d+\.\d+\s*\(")
    if len(pattern.findall(text)) > 0: hs_code.append(None)
    else: pattern = re.compile("\d+\.\d+\.\d+")
    if len(pdf_name) > len(hs_code): hs_code.append(pattern.findall(text)[0])

    # UPAS tenor
    val = get_data_btn(text, "DESPITE", "AT MATURITY")
    pattern = re.compile("\d+")
    try: upas_tenor_1.append(pattern.findall(val)[0])
    except: upas_tenor_1.append(None)
    try: upas_tenor_2.append(pattern.findall(val)[1])
    except: upas_tenor_2.append(None)

    # text
    text = get_data_btn(extract_text(f), "GOODS:", "DOCUMENTS REQUIRED:")
    text = re.sub("Page:\d+\s/\s\d+", " ", text)
    text = re.sub(" +", " ", text).split("\n")

    # material name
    material = []
    for t in text: 
        pattern = re.compile("^(?!\s*QUAN).+AT THE RATE")
        vals = pattern.findall(t)
        for v in vals: material.append(v[0:-11].strip())
    for t in text:
        if len(material) > 0: break
        pattern = re.compile("\+.+")
        vals = pattern.findall(t)
        for v in vals: material.append(v.strip())
    mat = ''
    for m in material:
        pattern = re.compile("[A-Z0-9].+")
        mat = mat + pattern.findall(m)[0] + ', '
    material_name.append(mat[:-2])

    # accumulate
    df_hsbc = pd.DataFrame()
    df_hsbc['pdf_name'] = pdf_name
    df_hsbc['dc_no'] = dc_no
    df_hsbc['dc_curr'] = dc_curr
    df_hsbc['dc_amt'] = dc_amt
    df_hsbc['beneficiary'] = beneficiary
    df_hsbc['issue_date'] = issue_date
    df_hsbc['issue_date_refined'] = [datetime.strptime(idate, "%y%m%d").strftime("%d-%b-%y") for idate in issue_date]
    df_hsbc['insurance_no'] = insurance_no
    df_hsbc['insurance_date'] = insurance_date
    df_hsbc['insurance_date_refined'] = [datetime.strptime(idate, "%d%b%Y").strftime("%d-%b-%y") for idate in insurance_date]
    df_hsbc['material_name'] = material_name
    df_hsbc['payment_term'] = payment_term
    df_hsbc['bb_ref'] = bb_ref
    df_hsbc['hs_code_importer'] = hs_code_importer
    df_hsbc['hs_code_exporter'] = hs_code_exporter
    df_hsbc['hs_code'] = hs_code
    df_hsbc['upas_tenor_1'] = upas_tenor_1
    df_hsbc['upas_tenor_2'] = upas_tenor_2
    df_hsbc['inv_no'] = inv_no
    df_hsbc['bank'] = 'HSBC'

    # return 
    return df_hsbc


In [4]:
# BBL
def parse_bbl(f):

    # fetch datapoint
    def get_data_btn(text_str, sub1, sub2):
        text_str = text_str.replace(sub1, "*").replace(sub2, "*")
        datapoint = text_str.split("*")[1].strip()
        return datapoint

    # scrape
    pdf_name = []
    dc_no = []
    dc_curr = []
    dc_amt = []
    beneficiary = []
    issue_date = []
    insurance_no = []
    insurance_date = []
    material_name = []
    payment_term = []
    bb_ref = []
    inv_no = []
    hs_code_importer = []
    hs_code_exporter = []
    hs_code = []
    upas_tenor_1 = []
    upas_tenor_2 = []
    
    # doc name
    val = f.split("\\")[-1]
    print("Parsing: " + val)
    pdf_name.append(val)

    # all text
    text = extract_text(f)
    
    # dc no.
    val = get_data_btn(text, "Documentary Credit Number", "Date of Issue")
    dc_no.append(val)
    
    # dc value
    pattern = re.compile("(?:USD|EUR|BDT|INR)\s*[\d\.,]+")
    val = pattern.findall(text)[0]
    # currency
    pattern = re.compile("[A-Z]+")
    dc_curr.append(pattern.findall(val)[0])
    # amount
    pattern = re.compile("[\d\.,]+")
    dc_amt.append(pattern.findall(val)[0])
    
    # beneficiary
    val = get_data_btn(text, "Beneficiary", "Currency Code, Amount")
    pattern = re.compile(".+(?:LTD|LIMITED)")
    try: beneficiary.append(pattern.findall(val)[0])
    except: beneficiary.append(val.split("\n")[0])
    
    # issue date
    val = get_data_btn(text, "Date of Issue", "Applicable Rules")
    issue_date.append(val)
    
    # insurance
    val = get_data_btn(text, "NUMBER AND DATE ", " AND A COPY OF")
    # insurance no.
    pattern = re.compile("UIC[\w/\-\(\s]+[\)0-9]\s")
    insurance_no.append(pattern.findall(val)[0].strip())
    # insurance date
    val = val.split()[-1]
    insurance_date.append(val)
    
    # BB ref.
    val = get_data_btn(text, "BB. TIN       : ", "CC. BIN")
    bb_ref.append(val)
    
    # UPAS tenor
    val = get_data_btn(text, "1.DESPITE THE DC TENOR", "Sender to Receiver Information").split("\n")
    pattern = re.compile("\d+")
    try: upas_tenor_1.append(pattern.findall(val[0])[0])
    except: upas_tenor_1.append(None)
    try: upas_tenor_2.append(pattern.findall(val[1])[0])
    except: upas_tenor_2.append(None)
    
    # invoice no. 
    val = get_data_btn(text, "INVOICE NUMBER", "PRICE/DELIVERY TERMS")
    try: inv_no.append(val.split("DATED")[0].strip("\n "))
    except: inv_no.append(None)
        
    # payment term
    val = get_data_btn(text, "PRICE/DELIVERY TERMS:", "INCOTERMS 2020")
    try: payment_term.append(val)
    except: payment_term.append(None)
        
    # material name
    mat = ""
    val = get_data_btn(text, "Description of Goods and/or Services", "AND OTHER DETAILS").split("\n.\n")
    for v in val: mat = mat + v.split("\n")[0] + ", "
    material_name.append(mat[:-2])
    
    # text
    text = get_data_btn(text, "EE. H.S. CODE", ".\n3. COUNTRY OF ORIGIN,")
    text = text.replace(".", "").split("/")
        
    # HS code 
    code = []
    mode = []
    for t in text:
        # line
        pattern = re.compile("(?:PORT)*.*[0-9]{8}.*(?:PORT)*")
        try: val = pattern.findall(t)[0]
        except: continue
        # code
        pattern = re.compile("[0-9]{8}")
        vals = pattern.findall(val)
        for v in vals: code.append(v)
        # mode
        pattern = re.compile("[A-Z]{2}PORT")
        vals = pattern.findall(val)
        for v in vals: mode.append(v)
        # general
        mode_len = len(mode)
        for i in range(len(code)-len(mode)): 
            if mode_len == 0: mode.append("GENERAL")
            else: mode.append(mode[mode_len-1])
            
    # HS code - import/export/general
    imp = exp = gen = ''
    for i in range(0, len(code)):
        if mode[i] == 'IMPORT': imp = imp + code[i] + ', '
        elif mode[i] == 'EXPORT': exp = exp + code[i] + ', '
        else: gen = gen + code[i] + ', '    
    hs_code.append(gen[:-2])
    hs_code_importer.append(imp[:-2])
    hs_code_exporter.append(exp[:-2])
    
    # accumulate
    df_bbl = pd.DataFrame()
    df_bbl['pdf_name'] = pdf_name
    df_bbl['dc_no'] = dc_no
    df_bbl['dc_curr'] = dc_curr
    df_bbl['dc_amt'] = dc_amt
    df_bbl['beneficiary'] = beneficiary
    df_bbl['issue_date'] = issue_date
    df_bbl['issue_date_refined'] = [datetime.strptime(idate, "%y%m%d").strftime("%d-%b-%y") for idate in issue_date]
    df_bbl['insurance_no'] = insurance_no
    df_bbl['insurance_date'] = insurance_date
    df_bbl['insurance_date_refined'] = [datetime.strptime(idate, "%d-%b-%Y").strftime("%d-%b-%y") for idate in insurance_date]
    df_bbl['material_name'] = material_name
    df_bbl['payment_term'] = payment_term
    df_bbl['bb_ref'] = bb_ref
    df_bbl['hs_code_importer'] = hs_code_importer
    df_bbl['hs_code_exporter'] = hs_code_exporter
    df_bbl['hs_code'] = hs_code
    df_bbl['upas_tenor_1'] = upas_tenor_1
    df_bbl['upas_tenor_2'] = upas_tenor_2
    df_bbl['inv_no'] = inv_no
    df_bbl['bank'] = 'BBL'

    # return 
    return df_bbl


In [5]:
# SCB
def breach_scb(f, path):

    # password
    pdf = pikepdf.open(f, password="csg@7865", allow_overwriting_input=True)
    pdf.save(path + f.split("\\")[-1])

    # attachments
    doc = fitz.open(path + f.split("\\")[-1])
    name_dict = {}
    for item in doc.embfile_names(): name_dict[item] = doc.embfile_info(item)["filename"]

    # save
    for item, file in name_dict.items():
        if "ADV.pdf" in file:
            with open(path + file, "wb") as outfile: outfile.write(doc.embfile_get(item))
    doc.close()

def parse_scb(f):

    # fetch datapoint
    def get_data_btn(text_str, sub1, sub2):
        text_str = text_str.replace(sub1, "!").replace(sub2, "!")
        datapoint = text_str.split("!")[1].strip()
        return datapoint

    # scrape
    pdf_name = []
    dc_no = []
    dc_curr = []
    dc_amt = []
    beneficiary = []
    issue_date = []
    insurance_no = []
    insurance_date = []
    material_name = []
    payment_term = []
    bb_ref = []
    inv_no = []
    hs_code_importer = []
    hs_code_exporter = []
    hs_code = []
    upas_tenor_1 = []
    upas_tenor_2 = []
    
    # doc name
    name = f.split("\\")[-1]
    print("Parsing: " + name)
    pdf_name.append(name)

    # all text
    text = extract_text(f)

    # dc no.
    val = get_data_btn(text, "CREDIT NUMBER", ":31C")
    dc_no.append(val)

    # dc value
    pattern = re.compile("(?:USD|EUR|BDT|INR)\s*[\d\.,]+")
    val = pattern.findall(text)[0]
    # currency
    pattern = re.compile("[A-Z]+")
    dc_curr.append(pattern.findall(val)[0])
    # amount
    pattern = re.compile("[\d\.,]+")
    dc_amt.append(pattern.findall(val)[0])

    # beneficiary
    val = get_data_btn(text, ":59:      BENEFICIARY", ":32B:").split("\n")[0]
    beneficiary.append(val)

    # issue date
    val = get_data_btn(text, "DATE OF ISSUE", ":40E")
    issue_date.append(val)

    # insurance
    val = "UIC" + get_data_btn(text, "UIC", "\n.\n+ ")
    # insurance no.
    pattern = re.compile("UIC[\w/\-\(\s]+[\)0-9]\s")
    try: insurance_no.append(pattern.findall(val)[0].strip())
    except: insurance_no.append(None)
    # insurance date
    pattern = re.compile("[\d]{2}\.[\d]{2}\.[\d]{4}")
    try: insurance_date.append(pattern.findall(val)[0])
    except: insurance_date.append(None)

    # material name
    val = get_data_btn(text, "OR SERVICES\n+", "QUANTITY").replace("\n", " ")
    material_name.append(val)

    # payment term
    val = get_data_btn(text, "INCOTERMS ", ":46A").replace("\n", " ")
    payment_term.append(val)

    # BB ref.
    val = get_data_btn(text, "DC REFERENCE NUMBER:", " AND LC")
    bb_ref.append(val)

    # invoice no. 
    inv = ''
    val = get_data_btn(text, "46A:", "APPLICANT'S BIN")
    pattern = re.compile("[\S]+\sDATED")
    vals = pattern.findall(val)
    for v in vals: inv = inv + v[:-6] + ', '
    inv_no.append(inv[:-2])

    # UPAS tenor
    try: val = get_data_btn(text, "TENOR BEING", "COMPLYING DOCUMENTS")
    except: val = ""
    pattern = re.compile("[0-9]+")
    try: upas_tenor_1.append(pattern.findall(val)[0])
    except: upas_tenor_1.append(None)
    try: upas_tenor_2.append(pattern.findall(val)[1])
    except: upas_tenor_2.append(None)

    # text
    text = get_data_btn(text, "46A:", "INSURANCE COVER NOTE")
    text = text.replace(".", "").split("\n")

    # HS code 
    code = []
    mode = []
    for t in text:
        # line
        pattern = re.compile("(?:PORT)*.*CODE.*[0-9]{8}.*(?:PORT)*")
        try: val = pattern.findall(t)[0]
        except: continue
        # code
        pattern = re.compile("[0-9]{8}")
        vals = pattern.findall(val)
        for v in vals: code.append(v)
        # mode
        pattern = re.compile("[A-Z]{2}PORT")
        vals = pattern.findall(val)
        for v in vals: mode.append(v)
        # general
        mode_len = len(mode)
        for i in range(len(code)-len(mode)): 
            if mode_len == 0: mode.append("GENERAL")
            else: mode.append(mode[mode_len-1])

    # HS code - import/export/general
    imp = ''
    exp = ''
    gen = ''
    for i in range(0, len(code)):
        if mode[i] == 'IMPORT': imp = imp + code[i] + ', '
        elif mode[i] == 'EXPORT': exp = exp + code[i] + ', '
        else: gen = gen + code[i] + ', '    
    hs_code.append(gen[:-2])
    hs_code_importer.append(imp[:-2])
    hs_code_exporter.append(exp[:-2])

    # accumulate
    df_scb = pd.DataFrame()
    df_scb['pdf_name'] = pdf_name
    df_scb['dc_no'] = dc_no
    df_scb['dc_curr'] = dc_curr
    df_scb['dc_amt'] = dc_amt
    df_scb['beneficiary'] = beneficiary
    df_scb['issue_date'] = issue_date
    df_scb['issue_date_refined'] = [datetime.strptime(idate, "%y%m%d").strftime("%d-%b-%y") for idate in issue_date]
    df_scb['insurance_no'] = insurance_no
    df_scb['insurance_date'] = insurance_date
    df_scb['insurance_date_refined'] = [datetime.strptime(idate, "%d.%m.%Y").strftime("%d-%b-%y") for idate in insurance_date]
    df_scb['material_name'] = material_name
    df_scb['payment_term'] = payment_term
    df_scb['bb_ref'] = bb_ref
    df_scb['hs_code_importer'] = hs_code_importer
    df_scb['hs_code_exporter'] = hs_code_exporter
    df_scb['hs_code'] = hs_code
    df_scb['upas_tenor_1'] = upas_tenor_1
    df_scb['upas_tenor_2'] = upas_tenor_2
    df_scb['inv_no'] = inv_no
    df_scb['bank'] = 'SCB'

    # return 
    return df_scb


In [6]:
# PRB
def parse_prb(f):

    # fetch datapoint
    def get_data_btn(text_str, sub1, sub2):
        text_str = text_str.replace(sub1, "*").replace(sub2, "*")
        datapoint = text_str.split("*")[1].strip()
        return datapoint

    # scrape
    pdf_name = []
    dc_no = []
    dc_curr = []
    dc_amt = []
    beneficiary = []
    issue_date = []
    insurance_no = []
    insurance_date = []
    material_name = []
    payment_term = []
    bb_ref = []
    inv_no = []
    hs_code_importer = []
    hs_code_exporter = []
    hs_code = []
    upas_tenor_1 = []
    upas_tenor_2 = []
    
    # doc name
    val = f.split("\\")[-1]
    print("Parsing: " + val)
    pdf_name.append(val)

    # all text
    text = extract_text(f)
    collapsed_text = re.sub(" +", " ", text)
    compressed_text = text
    for s in["\n", " +"]: compressed_text = re.sub(s, " ", compressed_text)
    
    # dc no.
    val = get_data_btn(collapsed_text, "Credit Number : ", ":31C/Date of Issue")
    dc_no.append(val)
    
    # dc value
    pattern = re.compile("(?:USD|EUR|BDT|INR)\s*[\d\.,]+")
    val = pattern.findall(text)[0]
    # currency
    pattern = re.compile("[A-Z]+")
    dc_curr.append(pattern.findall(val)[0])
    # amount
    pattern = re.compile("[\d\.,]+")
    dc_amt.append(pattern.findall(val)[0])
    
    # beneficiary
    val = get_data_btn(collapsed_text, ":59/Beneficiary : ", ":32B/Currency Code, Amount")
    pattern = re.compile("[\w\W]+(?:LTD|LIMITED)")
    try: beneficiary.append(pattern.findall(val)[0])
    except: beneficiary.append(val.split("\n")[0])
    
    # issue date
    val = get_data_btn(collapsed_text, ":31C/Date of Issue : ", ":40E/Applicable Rules")
    issue_date.append(val)
    
    # insurance
    pattern = re.compile("UIC[\w/\-\(\s]+[\)0-9]\sDATED\s[\w\W]{10}")
    val = pattern.findall(compressed_text)[0].split(" DATED ")
    # insurance no.
    insurance_no.append(val[0])
    # insurance date
    insurance_date.append(val[1])
    
    # BB ref.
    bb_ref.append(None)
    
    # UPAS tenor
    val = get_data_btn(collapsed_text, ":42C/Drafts at … : ", ":42D/Drawee")
    pattern = re.compile("\d+")
    try: upas_tenor_1.append(pattern.findall(val)[0])
    except: upas_tenor_1.append(None)
    try: upas_tenor_2.append(pattern.findall(val)[1])
    except: upas_tenor_2.append(None)
    
    # invoice no. 
    val = get_data_btn(compressed_text, ":45A/Description of Goods and/or Service : ", ":46A/Documents Required")
    val = get_data_btn(val, "PROFORMA INVOICE NO.", "DATED")
    inv_no.append(val)
        
    # payment term
    val = get_data_btn(compressed_text, "TRADE TERM: ", ".. DESCRIPTION, QUALITY, QUANTITY")
    try: payment_term.append(val)
    except: payment_term.append(None)
        
    # material name
    mat = ""
    val = get_data_btn(compressed_text, ":45A/Description of Goods and/or Service : ", "TRADE TERM: ").split("..")[0:-1]
    for v in val: mat = mat + v.split(" QUANTITY")[0] + ", "
    material_name.append(mat[:-2])
    
    # text
    text = get_data_btn(collapsed_text, "B) ", "C) ")
    text = text.replace(".", "").split("\n")
    
    # HS code 
    code = []
    mode = []
    for t in text:
        # line
        pattern = re.compile("(?:PORT)*.*[0-9]{8}.*(?:PORT)*")
        try: val = pattern.findall(t)[0]
        except: continue
        # code
        pattern = re.compile("[0-9]{8}")
        vals = pattern.findall(val)
        for v in vals: code.append(v)
        # mode
        pattern = re.compile("[A-Z]{2}PORT")
        vals = pattern.findall(val)
        for v in vals: mode.append(v)
        # general
        mode_len = len(mode)
        for i in range(len(code)-len(mode)): 
            if mode_len == 0: mode.append("GENERAL")
            else: mode.append(mode[mode_len-1])
            
    # HS code - import/export/general
    imp = exp = gen = ''
    for i in range(0, len(code)):
        if mode[i] == 'IMPORT': imp = imp + code[i] + ', '
        elif mode[i] == 'EXPORT': exp = exp + code[i] + ', '
        else: gen = gen + code[i] + ', '    
    hs_code.append(gen[:-2])
    hs_code_importer.append(imp[:-2])
    hs_code_exporter.append(exp[:-2])
    
    # accumulate
    df_prb = pd.DataFrame()
    df_prb['pdf_name'] = pdf_name
    df_prb['dc_no'] = dc_no
    df_prb['dc_curr'] = dc_curr
    df_prb['dc_amt'] = dc_amt
    df_prb['beneficiary'] = beneficiary
    df_prb['issue_date'] = issue_date
    df_prb['issue_date_refined'] = ''
    df_prb['insurance_no'] = insurance_no
    df_prb['insurance_date'] = insurance_date
    df_prb['material_name'] = material_name
    df_prb['payment_term'] = payment_term
    df_prb['bb_ref'] = bb_ref
    df_prb['hs_code_importer'] = hs_code_importer
    df_prb['hs_code_exporter'] = hs_code_exporter
    df_prb['hs_code'] = hs_code
    df_prb['upas_tenor_1'] = upas_tenor_1
    df_prb['upas_tenor_2'] = upas_tenor_2
    df_prb['inv_no'] = inv_no
    df_prb['bank'] = 'PRB'

    # return 
    return df_prb


In [7]:
# LCs
fetch_read_lc('2023-09-01', '2032-12-31')

In [8]:
# accumulate HSBC
inc_docs = ""
df_hsbc = pd.DataFrame()
start_time = time.time()

# parse HSBC
files = glob.glob(r"C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/Emailed LCs/HSBC LCs/*-T02.PDF")
for f in reversed(files):
    try: df_hsbc = df_hsbc.append(parse_hsbc(f))
    except: inc_docs = inc_docs + f.split("\\")[-1] + ", "
df_hsbc = df_hsbc.reset_index(drop=True)
display(df_hsbc)

# analyse HSBC
email_hsbc_df = pd.DataFrame()
email_hsbc_df['Bank Name'] = ['HSBC']
email_hsbc_df['LCs Received'] = [df_hsbc.shape[0] + inc_docs.count(",")]
email_hsbc_df['LCs Parsed'] = [df_hsbc.shape[0]]
email_hsbc_df['LCs Incomplete'] = [inc_docs.count(",")]
email_hsbc_df['Incomplete LC Docs'] = [",".join(inc_docs.split(",", 3)[:3]) + ", ..."]
email_hsbc_df['Sec to Parse'] = [round(time.time() - start_time)]

Parsing: BDHSBC4B14290404-T02.PDF
Parsing: BDHSBC4B14287293-T02.PDF
Parsing: BDHSBC4B14287277-T02.PDF
Parsing: BDHSBC4B14287272-T02.PDF
Parsing: BDHSBC4B14287267-T02.PDF
Parsing: BDHSBC4B14287230-T02.PDF
Parsing: BDHSBC4B13273411-T02.PDF
Parsing: BDHSBC4B13273391-T02.PDF
Parsing: BDHSBC4B13273386-T02.PDF
Parsing: BDHSBC4B13273373-T02.PDF
Parsing: BDHSBC4B13273368-T02.PDF
Parsing: BDHSBC4B08226489-T02.PDF
Parsing: BDHSBC4B08226482-T02.PDF
Parsing: BDHSBC4B06198453-T02.PDF
Parsing: BDHSBC4B05191240-T02.PDF
Parsing: BDHSBC4B05185558-T02.PDF
Parsing: BDHSBC4B05185553-T02.PDF
Parsing: BDHSBC4B05185548-T02.PDF
Parsing: BDHSBC4B05185533-T02.PDF
Parsing: BDHSBC4B05185528-T02.PDF
Parsing: BDHSBC4B05185505-T02.PDF
Parsing: BDHSBC4B05185500-T02.PDF
Parsing: BDHSBC4B05185495-T02.PDF
Parsing: BDHSBC4B01151669-T02.PDF
Parsing: BDHSBC4A31138045-T02.PDF
Parsing: BDHSBC4A31138024-T02.PDF
Parsing: BDHSBC4A31137964-T02.PDF
Parsing: BDHSBC4A31137356-T02.PDF
Parsing: BDHSBC4A31137351-T02.PDF
Parsing: BDHSB

Parsing: BDHSBC3K06241155-T02.PDF
Parsing: BDHSBC3K06241150-T02.PDF
Parsing: BDHSBC3K06241145-T02.PDF
Parsing: BDHSBC3K06241140-T02.PDF
Parsing: BDHSBC3K05229581-T02.PDF
Parsing: BDHSBC3K05227505-T02.PDF
Parsing: BDHSBC3K02206152-T02.PDF
Parsing: BDHSBC3K02206002-T02.PDF
Parsing: BDHSBC3K01198040-T02.PDF
Parsing: BDHSBC3J31185802-T02.PDF
Parsing: BDHSBC3J31185786-T02.PDF
Parsing: BDHSBC3J31184970-T02.PDF
Parsing: BDHSBC3J31181152-T02.PDF
Parsing: BDHSBC3J31179398-T02.PDF
Parsing: BDHSBC3J30170498-T02.PDF
Parsing: BDHSBC3J30165533-T02.PDF
Parsing: BDHSBC3J29156155-T02.PDF
Parsing: BDHSBC3J29155065-T02.PDF
Parsing: BDHSBC3J29152067-T02.PDF
Parsing: BDHSBC3J29149501-T02.PDF
Parsing: BDHSBC3J29149496-T02.PDF
Parsing: BDHSBC3J29149491-T02.PDF
Parsing: BDHSBC3J29149486-T02.PDF
Parsing: BDHSBC3J26134440-T02.PDF
Parsing: BDHSBC3J25116618-T02.PDF
Parsing: BDHSBC3J25115277-T02.PDF
Parsing: BDHSBC3J25115272-T02.PDF
Parsing: BDHSBC3J23105691-T02.PDF
Parsing: BDHSBC3J23105372-T02.PDF
Parsing: BDHSB

,pdf_name,dc_no,dc_curr,dc_amt,beneficiary,issue_date,issue_date_refined,insurance_no,insurance_date,insurance_date_refined,material_name,payment_term,bb_ref,hs_code_importer,hs_code_exporter,hs_code,upas_tenor_1,upas_tenor_2,inv_no,bank
0,BDHSBC4A25075438-T02.PDF,DPCDAK400438,USD,"41333,60",UNILEVER INDIA EXPORTS LIMITED,240125,25-Jan-24,UIC/CZO/MC-0004/01/2024-(CO-INS),22JAN2024,22-Jan-24,,None,BANK DC NO. 0000285924020290,2104.10.00,2104.10.10,None,180,None,None,HSBC
1,BDHSBC4A24068705-T02.PDF,DPCDAK400423,USD,"40972,00",UNILEVER INDIA EXPORTS LIMITED,240124,24-Jan-24,UIC/CZO/MC-0086/01/2024-(CO-INS),21JAN2024,21-Jan-24,,None,BANK DC NO. 0000285924020281,2104.10.00,2104.10.10,None,180,None,None,HSBC
2,BDHSBC4A04863380-T02.PDF,DPCDAK400064,USD,"11450,00",SARAF FOODS LTD,240104,04-Jan-24,UIC/CZO/MC-2093/12/2023-(CO-INS),20DEC2023,20-Dec-23,1000.00 KG FREEZE DRIED SWEET CORN,"CFR CHATTOGARM SEAPORT, BANGLADESH",None,0712.90.10,0712.90.90,None,180,30,INVOICE NO.9412300265,HSBC
3,BDHSBC3K09285756-T02.PDF,DPCDAK312292,EUR,"2309,38",UNILEVER EUROPE BV WEENA 455 NL - 3013 AL- ROT...,231109,09-Nov-23,UIC/CZO/MC-1846/10/2023-(CO-INS),23OCT2023,23-Oct-23,400.00 KG BASE CHICKEN EF HALAL AS PER BENEFIC...,FCA HEILBRONN,None,None,None,3302.90.00,180,90,INVOICE NO.7801753185,HSBC
4,BDHSBC3K08269895-T02.PDF,DPCDAK312535,USD,"83596,80",LUBRIZOL ADVANCED MATERIALS EUROPE BV NIJVERHE...,231108,08-Nov-23,UIC/CZO/MC-1887/11/2023-(CO-INS),01NOV2023,01-Nov-23,MERQUAT 100 POLYMER,"CFR CHATTOGRAM SEAPORT, BANGLADESH",None,None,None,3911.90.00,180,None,INVOICE NO. 156466780,HSBC
5,BDHSBC3K06246318-T02.PDF,DPCDAK312445,USD,"52160,00",SARAF FOODS LTD,231106,06-Nov-23,UIC/CZO/MC-1866/10/2023-(CO-INS),30OCT2023,30-Oct-23,"FREEZE DRIED CHICKEN 1200KG, FREEZE DRIED SWEE...","CFR CHATTOGRAM PORT, BANGLADESH",None,0712.90.10,0712.90.90,None,180,30,None,HSBC
6,BDHSBC3K05229581-T02.PDF,DPCDAK312446,USD,"33626,00",UNILEVER INDIA EXPORTS LIMITED,231105,05-Nov-23,UIC/CZO/MC-1879/10/2023-(CO-INS),31OCT2023,31-Oct-23,BASE PWD FOR SWEET CORN CHICKEN SP 5 KG,"CFR CHATTOGRAM PORT, BANGLADESH",BANK DC NO.0000285923024995,2104.10.00,2104.10.10,None,180,None,INVOICE NO.UIEL/F&B/2023/011,HSBC
7,BDHSBC3J30170498-T02.PDF,DPCDAK312289,USD,"6128,00",INTERNATIONAL FLAVOURS AND FRAGRANCES (THAILAN...,231030,30-Oct-23,UIC/CZO/MC-1845/10/2023-(CO- INS),23OCT2023,23-Oct-23,400.000 KGS CHICKEN FLAVOUR UL137445,"CPT DHAKA AIRPORT, BANGLADESH",None,3302.90.00,3302.10.90,None,180,60,INVOICE NO.9306102276,HSBC
8,BDHSBC3J29156155-T02.PDF,DPCDAK312272,USD,"18525,00",SARAF FOODS LTD,231029,29-Oct-23,UIC/CZO/MC-1844/10/2023-(CO-INS),23OCT2023,23-Oct-23,500.00 KGS OVEGI FREEZE DRIED CHICKEN,CPT DHAKA AIRPORT BANGLADESH,BANK DC NO.0000285923024946,1601.00.00,0210.99.00,None,180,30,INVOICE NO. 9412300218,HSBC
9,BDHSBC3J16032646-T02.PDF,DPCDAK311941,USD,"7355,00",SARAF FOODS LTD,231016,16-Oct-23,UIC/CZO/MC-1782/10/2023-(CO-INS),11OCT2023,11-Oct-23,500 KGS OVEGI FREEZE DRIED SWEET CORN,"CPT DHAKA AIRPORT, BANGLADESH",BANK DC NO.0000285923024838,0712.90.10,0712.90.90,None,180,30,None,HSBC


In [9]:
# breach SCB
files = glob.glob(r"C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/Emailed LCs/SCB LCs/*.pdf")
for f in reversed(files): breach_scb(f, "C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/Emailed LCs/SCB LCs/SCB Breached LCs/")

# accumulate SCB
inc_docs = ""
df_scb = pd.DataFrame()
start_time = time.time()
    
# parse SCB
files = glob.glob(r"C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/Emailed LCs/SCB LCs/SCB Breached LCs/*ADV.pdf")
for f in reversed(files):
    try: df_scb = df_scb.append(parse_scb(f))
    except: inc_docs = inc_docs + f.split("\\")[-1] + ", "
df_scb = df_scb.reset_index(drop=True)
display(df_scb)

# analyse SCB
email_scb_df = pd.DataFrame()
email_scb_df['Bank Name'] = ['SCB']
email_scb_df['LCs Received'] = [df_scb.shape[0] + inc_docs.count(",")]
email_scb_df['LCs Parsed'] = [df_scb.shape[0]]
email_scb_df['LCs Incomplete'] = [inc_docs.count(",")]
email_scb_df['Incomplete LC Docs'] = [",".join(inc_docs.split(",", 3)[:3]) + ", ..."]
email_scb_df['Sec to Parse'] = [round(time.time() - start_time)]

C:\Users\Shithi.Maitra\Anaconda3\lib\site-packages\pikepdf\_methods.py:887: UserWarning: A password was provided, but no password was needed to open this PDF.
  pdf = Pdf._open(


Parsing: 422010634539-ISS000-MT700-ADV.pdf
Parsing: 422010634389-ISS000-MT700-ADV.pdf
Parsing: 422010634370-ISS000-MT700-ADV.pdf
Parsing: 422010634218-ISS000-MT700-ADV.pdf
Parsing: 422010633674-ISS000-MT700-ADV.pdf
Parsing: 422010632988-ISS000-MT700-ADV.pdf
Parsing: 422010632096-ISS000-MT700-ADV.pdf
Parsing: 422010632096-AMD001-MT707-ADV.pdf
Parsing: 422010631872-ISS000-MT700-ADV.pdf
Parsing: 422010631872-AMD001-MT707-ADV.pdf
Parsing: 422010631863-ISS000-MT700-ADV.pdf
Parsing: 422010631097-ISS000-MT700-ADV.pdf
Parsing: 422010629803-ISS000-MT700-ADV.pdf
Parsing: 422010629796-ISS000-MT700-ADV.pdf
Parsing: 422010629796-AMD001-MT707-ADV.pdf
Parsing: 422010629670-ISS000-MT700-ADV.pdf
Parsing: 422010629670-AMD001-MT707-ADV.pdf
Parsing: 422010629518-ISS000-MT700-ADV.pdf
Parsing: 422010629518-AMD001-MT707-ADV.pdf
Parsing: 422010629509-ISS000-MT700-ADV.pdf
Parsing: 422010629509-AMD001-MT707-ADV.pdf
Parsing: 422010629493-ISS000-MT700-ADV.pdf
Parsing: 422010629484-ISS000-MT700-ADV.pdf
Parsing: 42

Parsing: 421011191169-ISS000-MT700-ADV.pdf
Parsing: 421011191169-AMD002-MT707-ADV.pdf
Parsing: 421011191169-AMD001-MT707-ADV.pdf
Parsing: 421011191150-ISS000-MT700-ADV.pdf
Parsing: 421011191123-ISS000-MT700-ADV.pdf
Parsing: 421011191123-AMD001-MT707-ADV.pdf
Parsing: 421011191114-ISS000-MT700-ADV.pdf
Parsing: 421011191105-ISS000-MT700-ADV.pdf
Parsing: 421011190865-ISS000-MT700-ADV.pdf
Parsing: 421011190758-ISS000-MT700-ADV.pdf
Parsing: 421011190614-ISS000-MT700-ADV.pdf
Parsing: 421011190589-ISS000-MT700-ADV.pdf
Parsing: 421011190222-ISS000-MT700-ADV.pdf
Parsing: 421011190115-ISS000-MT700-ADV.pdf
Parsing: 421011189911-ISS000-MT700-ADV.pdf
Parsing: 421011189895-ISS000-MT700-ADV.pdf
Parsing: 421011189519-ISS000-MT700-ADV.pdf
Parsing: 421011189181-ISS000-MT700-ADV.pdf
Parsing: 421011189181-AMD001-MT707-ADV.pdf
Parsing: 421011188618-ISS000-MT700-ADV.pdf
Parsing: 421011188609-ISS000-MT700-ADV.pdf
Parsing: 421011188574-ISS000-MT700-ADV.pdf
Parsing: 421011188306-ISS000-MT700-ADV.pdf
Parsing: 42

,pdf_name,dc_no,dc_curr,dc_amt,beneficiary,issue_date,issue_date_refined,insurance_no,insurance_date,insurance_date_refined,material_name,payment_term,bb_ref,hs_code_importer,hs_code_exporter,hs_code,upas_tenor_1,upas_tenor_2,inv_no,bank
0,422010632096-ISS000-MT700-ADV.pdf,422010632096-G,EUR,"8653,72",ENDRESS HAUSER INTERNATIONAL AG,240110,10-Jan-24,UIC/CZO/MC-2123/12/2023-(CO-INS),27.12.2023,27-Dec-23,"TEMPERATURE MEASURING INSTRUMENT,COMPRESSION F...","2020 CPT HAZRAT SHAHJALAL INT. AIRPORT, DHAKA,...",249424020020,"90251900, 73072910, 90268020",,,57,50,,SCB
1,422010631097-ISS000-MT700-ADV.pdf,422010631097-G,USD,"1000,",FORBES MARSHALL PVT.LTD.,240110,10-Jan-24,UIC/CZO/MC-2119/12/2023-(CO-INS),27.12.2023,27-Dec-23,PH SENSOR(INSTRUMENTS FOR CHEMICAL ANALYSIS) M...,"2020 CPT DHAKA AIRPORT, BANGLADESH",249424020016,,,90282000,57,50,FMBD/NH/UBL/23/10,SCB
2,422010629803-ISS000-MT700-ADV.pdf,422010629803-G,USD,"22728,08",NILSAN NISHOTECH SYSTEMS PVT. LTD,231221,21-Dec-23,UIC/CZO/MC-2065/12/2023-(CO-INS),13.12.2023,13-Dec-23,"1. PARASITIC PUMP 323U/D ANALOGUE, MANUAL CONT...","2020 CPT DHAKA AIRPORT, BANGLADESH",249423020515,"84131900, 90268000, 90258000",84212190,,57,50,NNSPL/PI/066/2023-24,SCB
3,422010629796-ISS000-MT700-ADV.pdf,422010629796-G,USD,"53120,98",GEA FLOW COMPONENTS INDIA,231221,21-Dec-23,UIC/CZO/MC-2082/12/2023-(CO-INS),17.12.2023,17-Dec-23,"SS31GL PIPE 3A STANDARD SIZE76.20MM (3"") - COO...",: DOCUMENTS REQUIRED + BENEFICIARY'S SIGNED I...,249423020514,,,"73043100, 73072900, 90328900",57,50,90105986,SCB
4,422010629670-ISS000-MT700-ADV.pdf,422010629670-G,USD,"3325,",AAMIR UNIVERSAL TRADING CENTRE,231228,28-Dec-23,UIC/CZO/MC-2053/12/2023-(CO-INS),12.12.2023,12-Dec-23,SPARE PARTS OF PUMP,"2020 CPT DHAKA AIRPORT, BANGLADESH",249423020512,,,"84828000, 73181600, 40169300, 40169990",57,50,FPI/AUTC/PFI-02/2,SCB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,421011186763-ISS000-MT700-ADV.pdf,421011186763-G,USD,"34344,","GODREJ INDUSTRIES LTD, 2ND FLR,",231009,09-Oct-23,UIC/CZO/MC-1723/09/2023-(CO-INS),27.09.2023,27-Sep-23,CETO STEARYL ALCOHOL GINOL 1618 TA NDPE (MB),"2020 CFR CHATTOGRAM SEAPORT, BANGLADESH",248623023486,38237000,38237090,,90,57,3000003762,SCB
162,421011186451-ISS000-MT700-ADV.pdf,421011186451-G,USD,"36192,",PT UNILEVER OLEOCHEMICAL INDONESIA,231001,01-Oct-23,UIC/CZO/MC-1720/09/2023-(CO-INS),26.09.2023,26-Sep-23,MYRISTIC ACID (C14) MB,"2020 CFR CHATTOGRAM SEAPORT, BANGLADESH",248623023482,,,29159000,30,57,2023/PI/09-349,SCB
163,421011186380-ISS000-MT700-ADV.pdf,421011186380-G,USD,"45780,",INTERNATIONAL FLAVOURS AND,230927,27-Sep-23,UIC/CZO/MC-1721/09/2023-(CO-INS),26.09.2023,26-Sep-23,MIXTURE OF ODORIFEROUS SUBSTANCE: WHITE RUBY 1...,"2020 EXW SRICITY, INDIA",248623023480,33029000,33029011,,120,57,9909104120,SCB
164,421011186139-ISS000-MT700-ADV.pdf,421011186139-G,USD,"17730,",GFF EXPORTS,230926,26-Sep-23,UIC/CZO/MC-1718/09/2023-(CO-INS),25.09.2023,25-Sep-23,BUTYLENE GLYCOL BD,"2020 CFR CHATTOGRAM SEAPORT, BANGLADESH",248623023443,29053900,29053990,,57,50,PI-G/01/23-24,SCB


In [10]:
# accumulate BBL
inc_docs = ""
df_bbl = pd.DataFrame()
start_time = time.time()

# parse BBL
files = glob.glob(r"C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/Emailed LCs/BBL LCs/*SWIFT*.PDF")
for f in reversed(files):
    try: df_bbl = df_bbl.append(parse_bbl(f))
    except: inc_docs = inc_docs + f.split("\\")[-1] + ", "
df_bbl = df_bbl.reset_index(drop=True)
display(df_bbl)

# analyse BBL
email_bbl_df = pd.DataFrame()
email_bbl_df['Bank Name'] = ['BBL']
email_bbl_df['LCs Received'] = [df_bbl.shape[0] + inc_docs.count(",")]
email_bbl_df['LCs Parsed'] = [df_bbl.shape[0]]
email_bbl_df['LCs Incomplete'] = [inc_docs.count(",")]
email_bbl_df['Incomplete LC Docs'] = [",".join(inc_docs.split(",", 3)[:3]) + ", ..."]
email_bbl_df['Sec to Parse'] = [round(time.time() - start_time)]

Parsing: Payment swift copy.pdf
Parsing: F08102314517D001_SWIFT_071557613.pdf
Parsing: 308524020743_SWIFT_162403667.pdf
Parsing: 308524020743_SWIFT_141602757.pdf
Parsing: 308524020742_SWIFT_162403605.pdf
Parsing: 308524020742_SWIFT_142001646.pdf
Parsing: 308524020720_SWIFT_155159845.pdf
Parsing: 308524020719_SWIFT_155159922.pdf
Parsing: 308524020718_SWIFT_154802717.pdf
Parsing: 308524020465_SWIFT_202400378.pdf
Parsing: 308524020465_SWIFT_191200671.pdf
Parsing: 308524020416_SWIFT_190401942.pdf
Parsing: 308524020379_SWIFT_175600252.pdf
Parsing: 308524020153_SWIFT_192359640.pdf
Parsing: 308524020153_SWIFT_142757048.pdf
Parsing: 308524020150_SWIFT_194359464.pdf
Parsing: 308524020150_SWIFT_142404734.pdf
Parsing: 308524020127_SWIFT_200003063.pdf
Parsing: 308524020127_SWIFT_130404009.pdf
Parsing: 308524020126_SWIFT_200002997.pdf
Parsing: 308524020126_SWIFT_161602451.pdf
Parsing: 308524020124_SWIFT_200002931.pdf
Parsing: 308524020124_SWIFT_195958268.pdf
Parsing: 308524020096_SWIFT_195959922.pd

Parsing: 308523021845_SWIFT_185202508.pdf
Parsing: 308523021845_SWIFT_125203617.pdf
Parsing: 308523021845_SWIFT_125203550.pdf
Parsing: 308523021845_SWIFT_070359466.pdf
Parsing: 308523021807_SWIFT_200358011.pdf
Parsing: 308523021785_SWIFT_194004224.pdf
Parsing: 308523021785_SWIFT_145204640.pdf
Parsing: 308523021783_SWIFT_205158358.pdf
Parsing: 308523021783_SWIFT_202401036.pdf
Parsing: 308523021783_SWIFT_201600439.pdf
Parsing: 308523021783_SWIFT_155159594.pdf
Parsing: 308523021783_SWIFT_154403614.pdf
Parsing: 308523021728_SWIFT_204358541.pdf
Parsing: 308523021716_SWIFT_211602000.pdf
Parsing: 308523021715_SWIFT_214357607.pdf
Parsing: 308523021715_SWIFT_210800363.pdf
Parsing: 308523021715_SWIFT_205202335.pdf
Parsing: 308523021715_SWIFT_204403425.pdf
Parsing: 308523021715_SWIFT_203200105.pdf
Parsing: 308523021715_SWIFT_202403157.pdf
Parsing: 308523021715_SWIFT_202400216.pdf
Parsing: 308523021715_SWIFT_202400194.pdf
Parsing: 308523021715_SWIFT_202400161.pdf
Parsing: 308523021715_SWIFT_201958

,pdf_name,dc_no,dc_curr,dc_amt,beneficiary,issue_date,issue_date_refined,insurance_no,insurance_date,insurance_date_refined,material_name,payment_term,bb_ref,hs_code_importer,hs_code_exporter,hs_code,upas_tenor_1,upas_tenor_2,inv_no,bank
0,308524020720_SWIFT_155159845.pdf,308524020720,USD,"762425,",FARABI PETROCHEMICALS COMPANY,240211,11-Feb-24,UIC/CZO/MC-0154/02/2024(CO-INS),06-FEB-2024,06-Feb-24,LINEAR ALKYL BENZENE(FARALAB),"CFR CHATTOGRAM SEA PORT,BANGLADESH",149691242037,38170090,38170000,,270,90,971000071,BBL
1,308524020719_SWIFT_155159922.pdf,308524020719,USD,"762425,",FARABI PETROCHEMICALS COMPANY,240211,11-Feb-24,UIC/CZO/MC-0153/02/2024(CO-INS),06-FEB-2024,06-Feb-24,LINEAR ALKYL BENZENE(FARALAB),"CFR CHATTOGRAM SEA PORT,BANGLADESH",149691242037,38170090,38170000,,270,90,971000070,BBL
2,308524020465_SWIFT_202400378.pdf,308524020465,USD,"7560,",PRIVATE LIMITED,240125,25-Jan-24,UIC/CZO/MC-0118/01/2024(CO-INS),25-JAN-2024,25-Jan-24,HOSTAGEL WC,"CPT DHAKA INTERNATIONAL AIRPORT,BANGLADESH\nBY...",149691242037,34029090,39072990,,270,60,15998269,BBL
3,308524020416_SWIFT_190401942.pdf,308524020416,USD,"18845,",ROHA DYECHEM PVT.LTD,240123,23-Jan-24,UIC/CZO/MC-0100/01/2024(CO-INS),23-JAN-2024,23-Jan-24,IDACOL PATENT BLUE V,"CPT DHAKA AIRPORT,BANGLADESH BY AIR",149691242037,32041200,32041215,,270,None,16003791,BBL
4,308524020150_SWIFT_194359464.pdf,308524020150,USD,"773950,",FARABI PETROCHEMICALS COMPANY,240110,10-Jan-24,UIC/CZO/MC-0053/01/2024(CO-INS),10-JAN-2024,10-Jan-24,LINEAR ALKYL BENZENE(FARALAB),"CFR CHATTOGRAM SEA PORT,BANGLADESH",149691242037,,,38170000,270,90,971000033,BBL
5,308524020025_SWIFT_180001121.pdf,308524020025,USD,"3325,",KUSUMA PHARMA,240102,02-Jan-24,UIC/CZO/MC-0008/01/2024(CO-INS),02-JAN-2024,02-Jan-24,"ZINC CITRATE TRIHYDRATE (PURE), FOB VALUE: USD...","CPT DHAKA INTERNATIONAL AIRPORT,BANGLADESH\nBY...",149691242037,,,29181500,270,None,130,BBL
6,308523025846_SWIFT_125158857.pdf,308523025846,USD,"84420,",SURENDRA ENTERPRISE,231227,27-Dec-23,UIC/CZO/MC-2115/12/2023(CO-INS),26-DEC-2023,26-Dec-23,DOLOMITE POWDER.,"CFR CHATTOGRAM,BANGLADESH BY SEA",149691242037,25171090,25181000,,270,30,E/UBL/12251,BBL
7,308523021983_SWIFT_190756501.pdf,308523021983,USD,"402876,",A.R.STANCHEM PVT.LTD,230531,31-May-23,UIC/CZO/MC-1014/05/2023(CO-INS),28-MAY-2023,28-May-23,ORGANIC SURFACE ACTIVE AGENTS.,"CPT BENAPOLE, BANGLADESH BY ROAD INCOTERMS\n20...",149691242037,,,34023100,180,30,ARS/2324/PI/90035,BBL
8,308523021978_SWIFT_194759437.pdf,308523021978,USD,"78000,",SURENDRA ENTERPRISE,230530,30-May-23,UIC/CZO/MC-1015/05/2023(CO-INS),28-MAY-2023,28-May-23,DOLOMITE POWDER.,"CFR CHATTOGRAM,BANGLADESH BY SEA",149691242037,,,25181000,180,45,SE/UBL/05161,BBL


In [11]:
# accumulate PRB
inc_docs = ""
df_prb = pd.DataFrame()
start_time = time.time()

# parse PRB
files = glob.glob(r"C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/Emailed LCs/PRB LCs/*700.pdf")
for f in reversed(files):
    try: df_prb = df_prb.append(parse_prb(f))
    except: inc_docs = inc_docs + f.split("\\")[-1] + ", "
df_prb = df_prb.reset_index(drop=True)
display(df_prb)

# analyse PRB
email_prb_df = pd.DataFrame()
email_prb_df['Bank Name'] = ['PRB']
email_prb_df['LCs Received'] = [df_prb.shape[0] + inc_docs.count(",")]
email_prb_df['LCs Parsed'] = [df_prb.shape[0]]
email_prb_df['LCs Incomplete'] = [inc_docs.count(",")]
email_prb_df['Incomplete LC Docs'] = [",".join(inc_docs.split(",", 3)[:3]) + ", ..."]
email_prb_df['Sec to Parse'] = [round(time.time() - start_time)]

Parsing: DRAFT-845,730.00-700.pdf
Parsing: 166929700.pdf


""


In [12]:
# OP

# file
qry = '''select * from df_hsbc union all select * from df_bbl union all select * from df_scb /*union all select * from df_prb*/'''
df = duckdb.query(qry).df()
df.to_excel("LCs_parsed_test.xlsx", index=False)

# analysis
qry = '''select * from email_hsbc_df union all select * from email_scb_df union all select * from email_bbl_df /*union all select * from email_prb_df*/'''
email_df = duckdb.query(qry).df()
display(email_df)

,Bank Name,LCs Received,LCs Parsed,LCs Incomplete,Incomplete LC Docs,Sec to Parse
0,HSBC,374,27,347,"BDHSBC4B14290404-T02.PDF, BDHSBC4B14287293-T02...",83
1,SCB,271,166,105,"422010634539-ISS000-MT700-ADV.pdf, 42201063438...",136
2,BBL,289,9,280,"Payment swift copy.pdf, F08102314517D001_SWIFT...",34


In [13]:
# email
ol = win32com.client.Dispatch("outlook.application")
olmailitem = 0x0
newmail = ol.CreateItem(olmailitem)

# subject, recipients
newmail.Subject = 'Parsed LCs in Test'
# newmail.To = 'shithi.maitra@unilever.com'
newmail.To = 'ubl_tradealert@Unilever.com'
newmail.CC = 'mehedi.asif@unilever.com; asif.rezwan@unilever.com; md-ashiqur.akhand@unilever.com; shaik.hossen@unilever.com'

# body
newmail.HTMLbody = f'''
Dear concern,<br><br>
The service of automated parsing of datapoints from LCs is now developed and under testing. Please find (and verify) attached results from LCs shared <a href="mailto: ubl_tradealert@Unilever.com">@ubl_tradealert</a>.
''' + build_table(email_df, random.choice(['green_light', 'red_light', 'blue_light', 'grey_light', 'orange_light']), font_size='10px', text_align='left') + '''
Note that, the statistics presented above reflect LCs shared since 26-Sep-2023. More banks will be added to the test eventually. This is an auto email via <i>win32com</i>.<br><br>
Thanks,<br>
Shithi Maitra<br>
Asst. Manager, Cust. Service Excellence<br>
Unilever BD Ltd.<br>
'''

# attachment(s) 
folder = r"C:/Users/Shithi.Maitra/Unilever Codes/Ad Hoc/PR Prioritization Procurement/"
filename = folder + "LCs_parsed_test.xlsx"
newmail.Attachments.Add(filename)

# send
newmail.Send()